In [1]:
import pandas as pd, numpy as np
import zipfile

! whoami
! date

zmbc
Tue Sep 27 15:46:39 PDT 2022


In [2]:
# 2019 data does not work!
! mkdir -p ../data/raw/acs_2019_5yr
! [ ! -e ../data/raw/acs_2019_5yr/csv_pus.zip ] && wget https://www2.census.gov/programs-surveys/acs/data/pums/2019/5-Year/csv_pus.zip -P ../data/raw/acs_2019_5yr

In [3]:
! mkdir -p ../data/raw/acs_2018_5yr
! [ ! -e ../data/raw/acs_2018_5yr/csv_pus.zip ] && wget https://www2.census.gov/programs-surveys/acs/data/pums/2018/5-Year/csv_pus.zip -P ../data/raw/acs_2018_5yr

In [4]:
try:
    with zipfile.ZipFile('../data/raw/acs_2019_5yr/csv_pus.zip', 'r') as zip_ref:
        zip_ref.extractall('../data/raw/acs_2019_5yr')
except zipfile.BadZipFile:
    print('2019 5-year zip file malformed')

2019 5-year zip file malformed


In [5]:
try:
    with zipfile.ZipFile('../data/raw/acs_2018_5yr/csv_pus.zip', 'r') as zip_ref:
        zip_ref.extractall('../data/raw/acs_2018_5yr')
except zipfile.BadZipFile:
    print('2018 5-year zip file malformed')

2018 5-year zip file malformed


In [6]:
! mkdir -p ../data/raw/acs_2019_1yr
! [ ! -e ../data/raw/acs_2019_1yr/csv_pus.zip ] && wget https://www2.census.gov/programs-surveys/acs/data/pums/2019/1-Year/csv_pus.zip -P ../data/raw/acs_2019_1yr

In [7]:
with zipfile.ZipFile('../data/raw/acs_2019_1yr/csv_pus.zip', 'r') as zip_ref:
    zip_ref.extractall('../data/raw/acs_2019_1yr')

In [8]:
acs_2019_1yr = pd.concat([
    pd.read_csv('../data/raw/acs_2019_1yr/psam_pusa.csv'),
    pd.read_csv('../data/raw/acs_2019_1yr/psam_pusb.csv'),
])

In [9]:
acs_2019_1yr.to_csv('../data/acs_2019_1yr.csv')